## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Create a Neural Network class
class ThreeLayerNeuralNetwork:
    # Initialize the class
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate, activation_function, Optimizer): 
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate
        self.activation_function = activation_function
        self.Optimizer = Optimizer

    def change_learning_rate(self, learning_rate):
        self.learning_rate = learning_rate

    def change_activation_function(self, activation_function):
        self.activation_function = activation_function

    def change_Optimizer(self, Optimizer):
        self.Optimizer = Optimizer

    def change_hidden_nodes(self, hidden_nodes):
        self.hidden_nodes = hidden_nodes

    def _init_weight_matrices(self, input_nodes, hidden_nodes, output_nodes):
        # The Size of the Weight Matrix 1 is (hidden_nodes, input_nodes)
        self.weight_matrix_1 = np.random.normal(0.0, pow(self.hidden_nodes, -0.5), (self.hidden_nodes, self.input_nodes))
        # The Size of the Weight Matrix 2 is (output_nodes, hidden_nodes)
        self.weight_matrix_2 = np.random.normal(0.0, pow(self.output_nodes, -0.5), (self.output_nodes, self.hidden_nodes))

    def _init_bias_vectors(self, input_nodes, hidden_nodes, output_nodes):
        # The Size of the Bias Vector 1 is (hidden_nodes, 1)
        self.bias_vector_1 = np.random.normal(0.0, pow(self.hidden_nodes, -0.5), (self.hidden_nodes, 1))
        # The Size of the Bias Vector 2 is (output_nodes, 1)
        self.bias_vector_2 = np.random.normal(0.0, pow(self.output_nodes, -0.5), (self.output_nodes, 1))

    def _loss_function(self, y, y_hat):
        # The Loss Function is MSE
        return np.square(y - y_hat)
    
    def _forward_pass(self, input_vector):
        # The Size of the Input Vector is (input_nodes, 1)
        self.input_vector = input_vector
        # The Size of the Hidden Layer is (hidden_nodes, 1)
        self.hidden_layer = np.dot(self.weight_matrix_1, self.input_vector) + self.bias_vector_1
        print(self.hidden_layer)        
        # The Size of the Output Layer is (output_nodes, 1)
        self.output_layer = np.dot(self.weight_matrix_2, self.hidden_layer) + self.bias_vector_2       
        print(self.output_layer)
        print('==============================================================================================') 


    def Train(self, train_data):
        # Initialize the weights first
        self._init_weight_matrices(self.input_nodes, self.hidden_nodes, self.output_nodes)
        # Initialize the bias first
        self._init_bias_vectors(self.input_nodes, self.hidden_nodes, self.output_nodes)

        # Initialize the Optimizer
        if self.Optimizer == 'SGD':
            # Optimizer = SGD(self.learning_rate)
            pass

        # Initialize the Activation Function
        if self.activation_function == 'sigmoid':
            # Activation_Function = sigmoid()
            pass

        # Initialize the Loss Function
        if self.loss_function == 'MSE':
            # Loss_Function = MSE()
            pass

        # Test the Forward Pass, function
        self._forward_pass(train_data[0][0])



    def Test(self, data):
        pass

    def Predict(self, data_point):
        pass



## Load the Data

In [ ]:
# Load the Data
data = pd.read_csv('./Concrete_Data.csv', header=None)
# Remove the First Row
data = data.iloc[1:]
# Print the Head of the Data
print(data.head())

# Target Variable 
target_variable = data.iloc[:, -1]
# Input Variables
input_variables = data.iloc[:, 0:7]

# Convert both to numpy arrays
target_variable = np.array(target_variable, dtype=np.float32)
input_variables = np.array(input_variables, dtype=np.float32)

print(target_variable.shape)
print(input_variables.shape)

# Preprocess the Data
########################

########################

# Split the Data into Train and Test
# Randomly choose 70% and 30% of the data
Total_samples = target_variable.shape[0]
Train_percentage = 0.7

Train_Samples = 0.7*Total_samples
Test_Samples = 0.3*Total_samples

# Shuffle The Data
indices = np.arange(Total_samples)
np.random.shuffle(indices)

# Split this data
Train_indices = indices[:int(Train_Samples)]
Test_indices = indices[int(Train_Samples):]

X_Train = input_variables[Train_indices]
Y_Train = target_variable[Train_indices]
x_test = input_variables[Test_indices]
y_test = target_variable[Test_indices]


# Training the Neural Network

In [ ]:
# Create a Instance of The N.N class
Neural_Network = ThreeLayerNeuralNetwork(input_nodes=8, hidden_nodes=25, output_nodes=1, learning_rate=0.1, activation_function='sigmoid', Optimizer='SGD')
Neural_Network.Train(data)
